In [1]:
from pathlib import Path
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import trimesh

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
TRAIN_PATH = Path('MPI-FAUST/training/registrations')

In [3]:
def load_ply_mesh(p):
    """
    p: Path to .ply
    return: (V, F) with V:(n,3) float64, F:(m,3) int
    """
    m = trimesh.load_mesh(str(p), process=False)
    # trimesh는 faces가 (m,3), vertices가 (n,3)
    V = np.asarray(m.vertices, dtype=np.float64)
    F = np.asarray(m.faces, dtype=np.int32)
    return V, F

# 사용 가능한 파일 확인
files = sorted(TRAIN_PATH.glob("*.ply"))
print(f"#files: {len(files)}")
if files[:3]: 
    for f in files[:3]:
        print("e.g.", f.name)

#files: 100
e.g. tr_reg_000.ply
e.g. tr_reg_001.ply
e.g. tr_reg_002.ply


In [4]:
V, F = load_ply_mesh(files[0])
print("V shape:", V.shape, "F shape:", F.shape)
print("bbox min:", V.min(0), "bbox max:", V.max(0))
print("mean edge length ~", np.linalg.norm(V[F[:,0]]-V[F[:,1]], axis=1).mean())

V shape: (6890, 3) F shape: (13776, 3)
bbox min: [-0.37152582 -1.15803194 -0.07673693] bbox max: [0.42788532 0.61692756 0.3256287 ]
mean edge length ~ 0.016820639454223272


In [5]:
def to_o3d_mesh(V, F):
    m = o3d.geometry.TriangleMesh(
        o3d.utility.Vector3dVector(V),
        o3d.utility.Vector3iVector(F)
    )
    m.compute_vertex_normals()
    return m

# mesh = to_o3d_mesh(V, F)
# o3d.visualization.draw_geometries([mesh])  # 새 창으로 3D 뷰어

In [6]:
V_s, F_s = load_ply_mesh(files[0])
V_t, F_t = load_ply_mesh(files[1])

mesh_s = to_o3d_mesh(V_s, F_s)
mesh_t = to_o3d_mesh(V_t, F_t)

# 한쪽을 옆으로 이동해서 나란히 띄우기
offset = np.array([1.2*(V_s[:,0].ptp()), 0, 0])  # x축으로 bbox폭 만큼
mesh_t.translate(offset)

o3d.visualization.draw_geometries([mesh_s, mesh_t])

In [7]:
def to_mesh(V, F, colors=None):
    mesh = o3d.geometry.TriangleMesh(
        o3d.utility.Vector3dVector(V),
        o3d.utility.Vector3iVector(F)
    )
    mesh.compute_vertex_normals()
    if colors is not None:
        mesh.vertex_colors = o3d.utility.Vector3dVector(colors)
    return mesh

# def random_colormap(n):
#     cmap = plt.colormaps.get_cmap('hsv')  # 새 API
#     colors = cmap(np.linspace(0, 1, n))[:, :3]
#     return colors

# n_parts = 10  # 예: 신체 10영역
# labels = np.random.randint(0, n_parts, len(V_s))  # 예시: 랜덤 파트 라벨
# colors_src = random_colormap(n_parts)[labels]
colors_src = (V_s - V_s.min(0)) / (V_s.max(0) - V_s.min(0) + 1e-12)
mesh_src = to_mesh(V_s, F_s, colors_src)

In [8]:
n_s, n_t = len(V_s), len(V_t)

# 랜덤 하드 매핑 (source마다 하나의 target)
# vertex 개수가 다르면 일부는 중복 허용 (작은 쪽 기준)
idx_t = np.random.choice(n_t, size=n_s, replace=False)

# OT matrix T: (n_s, n_t)
T = np.zeros((n_s, n_t), dtype=np.float64)
T[np.arange(n_s), idx_t] = 1.0

In [9]:
def transfer_colors_soft(T, colors_src): 
    w = T.sum(0, keepdims=True) + 1e-12 
    colors_t = (T.T @ colors_src) / w.T 
    return np.clip(colors_t, 0, 1) 

colors_t = transfer_colors_soft(T, colors_src) 
mesh_t = to_mesh(V_t, F_t, colors_t)

In [12]:
offset = np.array([1.2*V_s[:, 0].ptp(), 0, 0])
mesh_t.translate(offset)
o3d.visualization.draw_geometries([mesh_src,mesh_t])